In [1]:
import os

import pymongo

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import pandas as pd

import re
from pprint import pprint
from datetime import datetime
import time

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, Date, DateTime, Boolean, BigInteger
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy import inspect



In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.youtube
#   collection.insert_one(test_programs)
#   collection = db[program_name]
#   collection.update_one({}, {'$set': kdhx_dict}, upsert=True)


In [3]:
rds_connection_string = "root:Password@127.0.0.1"
engine = create_engine(f'mysql://{rds_connection_string}')
engine.execute('CREATE DATABASE IF NOT EXISTS youtube_db')
engine.execute('USE youtube_db')
Base = declarative_base()


In [4]:
class Category_Ids(Base):
    __tablename__ = 'category_ids'
    id = Column(Integer, primary_key=True)
    category_name = Column(String(256))
    category_id = Column(String(256))
    
    def __init__(self, category_name, category_id):
        self.category_name = category_name
        self.category_id = category_id
        
        
    def __repr__(self):
        return f"id={self.id}, category_name={self.category_name}, category_id={category_id}"

In [5]:
class Topic_Ids(Base):
    __tablename__ = 'topic_ids'
    id = Column(Integer, primary_key=True)
    topic_name = Column(String(256))
    topic_id = Column(String(256))
    
    def __init__(self, topic_name):
        self.topic_name = topic_name
        self.topic_id = topic_id
        
    def __repr__(self):
        return f"id={self.id}, topic_name={self.topic_name} topic_id={topic_id}"

In [6]:
class Channel(Base):
    __tablename__ = 'channels'
    id = Column(Integer, primary_key=True)
    topic_ids_table_id = Column(Integer, ForeignKey('topic_ids.id'))
    channel_title = Column(String(256))
    channel_url = Column(String(256))
    channel_id = Column(String(256))
    subscribers = Column(BigInteger)
    video_count = Column(Integer)
    view_count = Column(BigInteger)
    country = Column(String(256))
    join_date = Column(Date)
    topic_ids = Column(String(256))
    cc_category = Column(String(256))
    thumbnail_url = Column(String(256))
    
    def __init__(self, topic_ids_table_id, channel_title, channel_url, channel_id, subscribers,\
                 video_count, view_count, country, join_date, topic_ids, cc_category, thumbnail_url):
        self.topic_ids_table_id = topic_ids_table_id
        self.channel_title = channel_title
        self.channel_url = channel_url
        self.channel_id = channel_id
        self.subscribers = subscribers
        self.video_count = video_count
        self.view_count = view_count
        self.country = country
        self.join_date = join_date
        self.join_dto = join_dto
        self.topic_ids = topic_ids
        self.cc_category = cc_category
        self.thumbnail_url = thumbnail_url
        
        
    def __repr__(self):
        return f"id={self.id}, channel_title={self.channel_title}, url={self.channel_url}"

In [7]:
class Videos(Base):
    __tablename__ = 'videos'
    id = Column(Integer, primary_key=True)
    channels_table_id = Column(Integer, ForeignKey('channels.id'))
    topic_ids_table_id = Column(Integer, ForeignKey('topic_ids.id'))
    category_ids_table_id = Column(Integer, ForeignKey('category_ids.id'))
    video_title = Column(String(256))
    view_count = Column(BigInteger)
    duration = Column(Integer) # https://stackoverflow.com/questions/15596753/how-do-i-get-video-durations-with-youtube-api-version-3
    like_count = Column(Integer)
    dislike_count = Column(Integer)
    favorite_count = Column(Integer)
    comment_count = Column(Integer)
    published_at = Column(DateTime)
    published_at_dto = Column(DateTime)
    tags = Column(String(256))
    dimension = Column(String(256))
    definition = Column(String(256))
    caption = Column(Boolean)
    licensedContent = Column(Boolean)
    license = Column(String(256))
    
    def __init__(self, channels_table_id, topic_ids_table_id, category_ids_table_id, video_title, view_count, duration,\
                 like_count, dislike_count, favorite_count, comment_count, published_at, published_at_dto, tags, dimension,\
                 definition, caption, licensedContent, license):
        self.channels_table_id = channels_table_id
        self.topic_ids_table_id = topic_ids_table_id
        self.category_ids_table_id = category_ids_table_id
        self.video_title = video_title
        self.view_count = view_count
        self.duration = duration
        self.like_count = like_count
        self.dislike_count = dislike_count
        self.favorite_count = favorite_count
        self.comment_count = comment_count
        self.published_at = published_at
        self.published_at_dto = published_at_dto
        self.tags = tags
        self.dimension = topic_ids
        self.definition = definition
        self.caption = caption
        self.licensedContent = licensedContent
        self.license = license
        
        
    def __repr__(self):
        return f"id={self.id}, video_title={self.video_title}, view_count={self.view_count}"

In [8]:
Session = sessionmaker(bind=engine)
sqlDb = Session()

In [9]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [10]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)


In [11]:
url = "http://www.channelcrawler.com/eng"
browser.visit(url)
browser.is_element_present_by_css("input#queryMinSubs", wait_time=1)
html = browser.html


In [12]:

minSubs = 10000000
driver = webdriver.Chrome()
driver.get(url)
sbox = driver.find_element_by_id('queryMinSubs')
sbox.send_keys(minSubs)
submit = driver.find_element_by_class_name('submitbutton')
submit.click()


In [13]:
select = Select(driver.find_element_by_id('sort-select'))
select.select_by_visible_text('Subscribers (descending)')

In [14]:
html = driver.page_source
channel_soup = bs(html, 'html.parser')
channel_divs = channel_soup.find_all('div', class_="channel")

In [15]:

list_10M_subs = []
next_button = driver.find_element_by_css_selector("a[rel='next']")
while next_button:
    html = driver.page_source
    collection = mongoDb.channel_crawler_html
    this_dict = {'channel_crawler_url': driver.current_url, 'channel_crawler_html':html}
    collection.insert_one(this_dict)
    channel_soup = bs(html, 'html.parser')
    channel_divs = channel_soup.find_all('div', class_="channel")
    for channel in channel_divs:
        collection = mongoDb.channel_crawler_data
        channel_dict = {}
        title_anchor = channel.find('a')
        try:
            country = channel.find('img')['title']
        except:
            country = ''
        smalls = channel.find_all('small')
        cc_category = smalls[0].get_text()
        small_text = smalls[1].get_text()
        subscribers = re.search(r'.+?(?=Subs)', small_text).group(0)
        subscribers = re.sub(r'\t', '', subscribers).strip()
        subscribers = int(re.sub(r',', '', subscribers))
        video_count = re.search(r'.+?(?=Video)', small_text).group(0) 
        video_count = re.sub(r'\t', '', video_count).strip()
        join_date = videos = re.search(r'(?<=Join Date:\s).*', small_text).group(0)
        join_date = re.sub(r'\t', '', join_date)
        try:
            join_dto = datetime.strptime(join_date, '%d.%m.%Y')
        except:
            join_dto = ''
        topic_ids_table_id = None
        channel_dict['topic_ids_table_id'] = topic_ids_table_id
        channel_title = title_anchor['title']
        channel_dict['channel_title'] = channel_title
        channel_url = title_anchor['href']
        channel_dict['channel_url'] = channel_url
        channel_id = re.search(r'(?<=channel/).*', channel_url).group(0)
        channel_dict['channel_id'] = channel_id
        channel_dict['subscribers'] = subscribers
        channel_dict['video_count'] = video_count
        view_count = None
        channel_dict['view_count'] = view_count
        channel_dict['country'] = country
        channel_dict['join_date'] = join_date        
        channel_dict['join_dto'] = join_dto
        topic_ids = None
        channel_dict['topic_ids'] = topic_ids
        channel_dict['cc_category'] = cc_category
        anchors = channel.find_all('a')
        thumbnail_url = anchors[1].find('img')['src']
        channel_dict['thumbnail_url'] = thumbnail_url
        list_10M_subs.append(channel_dict)
        collection.insert_one(channel_dict)
        sqlDb.add(Channel(topic_ids_table_id, channel_title, channel_url, channel_id, subscribers,\
                 video_count, view_count, country, join_date, topic_ids, cc_category, thumbnail_url))
        sqlDb.commit()
    try:
        next_button = driver.find_element_by_css_selector("a[rel='next']")
        next_button.click()
    except:
        break
    time.sleep(1)


C:\Users\soria\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\engine\default.py:509: Warning: (1292, "Incorrect date value: '29.04.2010' for column 'join_date' at row 1")
  cursor.execute(statement, parameters)
C:\Users\soria\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\engine\default.py:509: Warning: (1292, "Incorrect date value: '13.03.2006' for column 'join_date' at row 1")
  cursor.execute(statement, parameters)
C:\Users\soria\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\engine\default.py:509: Warning: (1292, "Incorrect date value: '15.11.2016' for column 'join_date' at row 1")
  cursor.execute(statement, parameters)
C:\Users\soria\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\engine\default.py:509: Warning: (1292, "Incorrect date value: '15.01.2007' for column 'join_date' at row 1")
  cursor.execute(statement, parameters)
C:\Users\soria\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\engine\default.py:509: Warning: (1292, "Incorrect date

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 1-4: ordinal not in range(256)

In [ ]:
subscribers

In [ ]:
list_10M_subs